# Export Pytorch to ONNX

In [1]:
import torch
from facenet.models.mtcnn import MTCNN, RNet, ONet, PNet
from facenet.models.inception_resnet_v1 import InceptionResnetV1
from face_config import FaceRecognitionConfig
import os
import cv2
import onnxruntime as ort
import numpy as np

/home/naiscorp/miniconda3/envs/face_rec/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = "cuda"
output_dir = "./models"
os.makedirs(output_dir, exist_ok=True)

## Test MTCNN Inference

In [ ]:
mtcnn = MTCNN(device=device)

input = cv2.imread("data/anh-son-tung-mtp-thumb.jpg")
faces = mtcnn(input)
print(faces.shape)
print(faces.device)

boxes, probs, points = mtcnn.detect(input, landmarks=True)
print(boxes.shape, probs.shape, points.shape)


<class 'torch.Tensor'>
torch.Size([3, 160, 160])
cpu
(1, 4) (1,) (1, 5, 2)


In [30]:
mtcnn = MTCNN(device=device)

input = cv2.imread("data/anh-son-tung-mtp-thumb.jpg")
faces = mtcnn(input)
print(faces.shape)
print(faces.device)

boxes, probs, points = mtcnn.detect(input, landmarks=True)
print(boxes.shape, probs.shape, points.shape)


<class 'torch.Tensor'>
torch.Size([3, 160, 160])
cpu
(1, 4) (1,) (1, 5, 2)


In [ ]:

pnet = PNet(pretrained=True)
pnet.eval().to(device)

dummy_input = torch.randn(1,3,256,256, device=device)
reg, probs = pnet(dummy_input)

print(reg.shape, probs.shape)


torch.Size([1, 4, 123, 123]) torch.Size([1, 2, 123, 123])


In [24]:
torch.onnx.export(
    pnet,
    (dummy_input,),
    os.path.join(output_dir, device + "_pnet.onnx"),
    export_params=True,
    opset_version=12,
    do_constant_folding=True,
    input_names=['input'],
    output_names=['bbox_regression', 'face_probability'],
    dynamic_axes={
        'input': {0: 'batch_size', 2: 'height', 3: 'width'},
        'bbox_regression': {0: 'batch_size', 2: 'height', 3: 'width'},
        'face_probability': {0: 'batch_size', 2: 'height', 3: 'width'}
    }
)


/tmp/ipykernel_148539/418914609.py:1: DeprecationWarning: You are using the legacy TorchScript-based ONNX export. Starting in PyTorch 2.9, the new torch.export-based ONNX exporter will be the default. To switch now, set dynamo=True in torch.onnx.export. This new exporter supports features like exporting LLMs with DynamicCache. We encourage you to try it and share feedback to help improve the experience. Learn more about the new export logic: https://pytorch.org/docs/stable/onnx_dynamo.html. For exporting control flow: https://pytorch.org/tutorials/beginner/onnx/export_control_flow_model_to_onnx_tutorial.html.
  torch.onnx.export(


In [25]:
rnet = RNet(pretrained=True)
rnet.eval().to(device)

dummy_input = torch.randn(1,3,24,24, device=device)
reg, probs = rnet(dummy_input)

print(reg.shape, probs.shape)


torch.Size([1, 4]) torch.Size([1, 2])


In [26]:
torch.onnx.export(
    rnet,
    (dummy_input,),
    os.path.join(output_dir, device + "_rnet.onnx"),
    export_params=True,
    opset_version=12,
    do_constant_folding=True,
    input_names=['input'],
    output_names=['bbox_regression', 'face_probability'],
    dynamic_axes={
        'input': {0: 'batch_size'},
        'bbox_regression': {0: 'batch_size'},
        'face_probability': {0: 'batch_size'}
    }
)

/tmp/ipykernel_148539/561709744.py:1: DeprecationWarning: You are using the legacy TorchScript-based ONNX export. Starting in PyTorch 2.9, the new torch.export-based ONNX exporter will be the default. To switch now, set dynamo=True in torch.onnx.export. This new exporter supports features like exporting LLMs with DynamicCache. We encourage you to try it and share feedback to help improve the experience. Learn more about the new export logic: https://pytorch.org/docs/stable/onnx_dynamo.html. For exporting control flow: https://pytorch.org/tutorials/beginner/onnx/export_control_flow_model_to_onnx_tutorial.html.
  torch.onnx.export(


In [27]:
onet = ONet(pretrained=True)
onet.eval().to(device)

dummy_input = torch.randn(1,3,48,48, device=device)
reg, points, probs = onet(dummy_input)

print(reg.shape, points.shape, probs.shape)


torch.Size([1, 4]) torch.Size([1, 10]) torch.Size([1, 2])


In [28]:
torch.onnx.export(
    onet,
    (dummy_input,),
    os.path.join(output_dir, device + "_onet.onnx"),
    export_params=True,
    opset_version=12,
    do_constant_folding=True,
    input_names=['input'],
    output_names=['bbox_regression', 'landmarks', 'face_probability'],  
    dynamic_axes={
        'input': {0: 'batch_size'},
        'bbox_regression': {0: 'batch_size'},
        'landmarks': {0: 'batch_size'},
        'face_probability': {0: 'batch_size'}
    }
)

/tmp/ipykernel_148539/2649185974.py:1: DeprecationWarning: You are using the legacy TorchScript-based ONNX export. Starting in PyTorch 2.9, the new torch.export-based ONNX exporter will be the default. To switch now, set dynamo=True in torch.onnx.export. This new exporter supports features like exporting LLMs with DynamicCache. We encourage you to try it and share feedback to help improve the experience. Learn more about the new export logic: https://pytorch.org/docs/stable/onnx_dynamo.html. For exporting control flow: https://pytorch.org/tutorials/beginner/onnx/export_control_flow_model_to_onnx_tutorial.html.
  torch.onnx.export(


In [34]:
inception = InceptionResnetV1(pretrained='casia-webface').eval().to(device)

inception_input = faces.unsqueeze(0).to(device)
output = inception(inception_input)
print(output.shape)

torch.onnx.export(
    inception,
    (inception_input,),
    os.path.join(output_dir, device + "_inception.onnx"),
    export_params=True,
    opset_version=12,
    do_constant_folding=True,
    input_names=['input'],
    output_names=['output'],
)

torch.Size([1, 512])


/tmp/ipykernel_148539/3821756062.py:7: DeprecationWarning: You are using the legacy TorchScript-based ONNX export. Starting in PyTorch 2.9, the new torch.export-based ONNX exporter will be the default. To switch now, set dynamo=True in torch.onnx.export. This new exporter supports features like exporting LLMs with DynamicCache. We encourage you to try it and share feedback to help improve the experience. Learn more about the new export logic: https://pytorch.org/docs/stable/onnx_dynamo.html. For exporting control flow: https://pytorch.org/tutorials/beginner/onnx/export_control_flow_model_to_onnx_tutorial.html.
  torch.onnx.export(


# Verify ONNX model

In [4]:
print(ort.get_available_providers())
print(ort.get_all_providers())

['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider']
['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'MIGraphXExecutionProvider', 'ROCMExecutionProvider', 'OpenVINOExecutionProvider', 'DnnlExecutionProvider', 'TvmExecutionProvider', 'VitisAIExecutionProvider', 'QNNExecutionProvider', 'NnapiExecutionProvider', 'VSINPUExecutionProvider', 'JsExecutionProvider', 'CoreMLExecutionProvider', 'ArmNNExecutionProvider', 'ACLExecutionProvider', 'DmlExecutionProvider', 'RknpuExecutionProvider', 'WebNNExecutionProvider', 'WebGpuExecutionProvider', 'XnnpackExecutionProvider', 'CANNExecutionProvider', 'AzureExecutionProvider', 'CPUExecutionProvider']


In [19]:
pnet_session = ort.InferenceSession(os.path.join(output_dir, device + "_pnet.onnx"), providers=['CPUExecutionProvider'])
rnet_session = ort.InferenceSession(os.path.join(output_dir, device + "_rnet.onnx"), providers=['CPUExecutionProvider'])
onet_session = ort.InferenceSession(os.path.join(output_dir, device + "_onet.onnx"), providers=['CPUExecutionProvider'])
inception_session = ort.InferenceSession(os.path.join(output_dir, device + "_inception.onnx"), providers=['CPUExecutionProvider'])


In [20]:
# List all input nodes
print('PNet')
for i, input in enumerate(pnet_session.get_inputs()):
    print(f"Input {i}: name={input.name}, shape={input.shape}, dtype={input.type}")

# List all output nodes
for i, output in enumerate(pnet_session.get_outputs()):
    print(f"Output {i}: name={output.name}, shape={output.shape}, dtype={output.type}")

print('RNet')
for i, input in enumerate(rnet_session.get_inputs()):
    print(f"Input {i}: name={input.name}, shape={input.shape}, dtype={input.type}")

for i, output in enumerate(rnet_session.get_outputs()):
    print(f"Output {i}: name={output.name}, shape={output.shape}, dtype={output.type}")

print('ONet')
for i, input in enumerate(onet_session.get_inputs()):
    print(f"Input {i}: name={input.name}, shape={input.shape}, dtype={input.type}")

for i, output in enumerate(onet_session.get_outputs()):
    print(f"Output {i}: name={output.name}, shape={output.shape}, dtype={output.type}")

print('Inception')
for i, input in enumerate(inception_session.get_inputs()):
    print(f"Input {i}: name={input.name}, shape={input.shape}, dtype={input.type}")

for i, output in enumerate(inception_session.get_outputs()):
    print(f"Output {i}: name={output.name}, shape={output.shape}, dtype={output.type}")
    

PNet
Input 0: name=input, shape=['batch_size', 3, 'height', 'width'], dtype=tensor(float)
Output 0: name=bbox_regression, shape=['batch_size', 4, 'height', 'width'], dtype=tensor(float)
Output 1: name=face_probability, shape=['batch_size', 2, 'height', 'width'], dtype=tensor(float)
RNet
Input 0: name=input, shape=['batch_size', 3, 24, 24], dtype=tensor(float)
Output 0: name=bbox_regression, shape=['batch_size', 4], dtype=tensor(float)
Output 1: name=face_probability, shape=['batch_size', 2], dtype=tensor(float)
ONet
Input 0: name=input, shape=['batch_size', 3, 48, 48], dtype=tensor(float)
Output 0: name=bbox_regression, shape=['batch_size', 4], dtype=tensor(float)
Output 1: name=landmarks, shape=['batch_size', 10], dtype=tensor(float)
Output 2: name=face_probability, shape=['batch_size', 2], dtype=tensor(float)
Inception
Input 0: name=input, shape=[1, 3, 160, 160], dtype=tensor(float)
Output 0: name=output, shape=[1, 512], dtype=tensor(float)


In [21]:
img = cv2.imread("data/anh-son-tung-mtp-thumb.jpg")
input = np.asarray(img).transpose(2, 0, 1)
input = np.expand_dims(input, 0).astype(np.float32)


In [22]:
input = np.random.randn(1,3,256,256).astype(np.float32)

In [23]:
outputs = pnet_session.run(None, {'input': input})
print(outputs[0].shape)
print(outputs[1].shape)

(1, 4, 123, 123)
(1, 2, 123, 123)


In [24]:
rnet_input = np.random.randn(12,3,24,24).astype(np.float32)
rnet_outputs = rnet_session.run(None, {'input': rnet_input})
print(rnet_outputs[0].shape)
print(rnet_outputs[1].shape)


(12, 4)
(12, 2)


In [26]:
onet_input = np.random.randn(1,3,48,48).astype(np.float32)
onet_outputs = onet_session.run(None, {'input': onet_input})
print(onet_outputs[0].shape)
print(onet_outputs[1].shape)
print(onet_outputs[2].shape)


(1, 4)
(1, 10)
(1, 2)


In [27]:
inception_input = np.random.randn(1,3,160,160).astype(np.float32)
inception_outputs = inception_session.run(None, {'input': inception_input})
print(inception_outputs[0].shape)

(1, 512)


# Build NCNN engine